In [48]:
!ollama pull deepseek-r1:1.5b

pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest 
pulling aabd4debf0c8... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 GB                         
pulling 369ca498f347... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  387 B                         
pulling 6e4c38e1172f... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.1 KB                         
pulling f4d24e9138dd... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  148 B                         
pulling a85fe2a2e58e... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [49]:
import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [50]:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [68]:
chan = Website("https://www.premierleague.com")
print(chan.title)
print(chan.text)

Premier League Live Scores, Stats & Blog | 2024/25
Skip to main navigation
Skip to main content
Club sites
Arsenal
Arsenal
Aston Villa
Aston Villa
AFC Bournemouth
AFC Bournemouth
Brentford
Brentford
Brighton & Hove Albion
Brighton & Hove Albion
Chelsea
Chelsea
Crystal Palace
Crystal Palace
Everton
Everton
Fulham
Fulham
Ipswich Town
Ipswich Town
Leicester
Leicester
Liverpool
Liverpool
Manchester City
Manchester City
Manchester United
Manchester United
Newcastle United
Newcastle United
Nottingham Forest
Nottingham Forest
Southampton
Southampton
Tottenham Hotspur
Tottenham Hotspur
West Ham United
West Ham United
Wolverhampton Wanderers
Wolverhampton Wanderers
Premier League
Home
Fixtures
Results
Tables
Transfers
Injury news
Stats
News
Video
Watch Live
Tickets
Clubs
Players
Awards
Fantasy
Fantasy
Fantasy Challenge
Fantasy Draft
Football & Community
Wider Football
PL Charitable Fund
Community
Youth Development
No Room for Racism
Inside Matters
Rainbow Laces
Poppy
More than a game
About
Over

In [64]:
system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignore text that might be navigation related.\
Respond in markdown."

In [78]:
def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too. If there are schedules, post the summary in a table\n\n"
    user_prompt += website.text
    return user_prompt

In [54]:
print(user_prompt_for(chan))

You are looking at a website titled Premier League Football News, Fixtures, Scores & Results
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Skip to main navigation
Skip to main content
Club sites
Arsenal
Arsenal
Aston Villa
Aston Villa
AFC Bournemouth
AFC Bournemouth
Brentford
Brentford
Brighton & Hove Albion
Brighton & Hove Albion
Chelsea
Chelsea
Crystal Palace
Crystal Palace
Everton
Everton
Fulham
Fulham
Ipswich Town
Ipswich Town
Leicester
Leicester
Liverpool
Liverpool
Manchester City
Manchester City
Manchester United
Manchester United
Newcastle United
Newcastle United
Nottingham Forest
Nottingham Forest
Southampton
Southampton
Tottenham Hotspur
Tottenham Hotspur
West Ham United
West Ham United
Wolverhampton Wanderers
Wolverhampton Wanderers
Premier League
Home
Fixtures
Results
Tables
Transfers
Injury news
Stats
News
Video
Watch Live
Tickets
Clubs
Players
Awards
F

In [79]:
def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [80]:
from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [81]:
def summarize(url):
    website = Website(url)
    response = ollama_via_openai.chat.completions.create(
        model="deepseek-r1:1.5b",
        messages = messages_for(website))
    return response.choices[0].message.content

In [82]:
summarize("https://www.premierleague.com")

'<think>\nOkay, so I\'m looking at this Premier League match where Manchester City played against Chelsea. Man City scored four goals before going down to a defensive struggle by Chelsea, which resulted in Man City winning 2-0. \n\nHmm, why does it mention something about "No Room for Racism"? I haven\'t heard that phrase mentioned elsewhere in the article. Maybe it\'s just a bit of extra fluff related to the team? I should check if Man City were ever racist before the match or after it. But from what I see, they weren’t racemic at the start, and not obviouslyracematic afterward either.\n\nWhat else am I supposed to figure out about that score? Well, let\'s break down what I know regarding this game. Both sides had strong teams earlier in the Premier League, Liverpool with 4-3 and Manchester City with 2-0 for a win. But Chelsea did concede easily, showing they were more dangerous defensively.\n\nManCity had goals from various midfielders: full-back Marko Robitaille, right-back Kevin De

In [83]:
def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [84]:
display_summary("https://www.premierleague.com")

<think>
Okay, so I'm trying to understand this article about the Premier League. The user already got some initial thoughts on it, but maybe they're feeling a bit unsure about how to structure their analysis or really grasp all the details. Let me break it down step by step.

First, I should consider what the Premier League is overall. It's a football competition in England, right? From 2013, they started with two teams on the field and added more later as they moved up the divisions. They're known for their competitive format throughout the season. That seems straightforward enough.

Now, looking at the article, it gives some stats like home scores, away scores, goals per match, etc., which are important to know if someone is a new fan or intern diving into data. It also mentions things like top players with specific statistics, team news, and transfer transfers. This could be useful for more in-depth studies later.

The article talks about clubs' stats but without much detail on why certain teams are performing better. Maybe it's just there to fill a space; I wonder if having that info would help someone make comparisons based on data or analysis.

Looking at the fixtures, they mention both home and away matches, which is helpful when planning travels or tracking form over time. I should note that because the Premier League can be quite involved in fixtures—maybe for casual sports fans to know.

When it comes to transfers, it's interesting that most transfers are up to the club, not just the top teams. That makes me curious about factors that influence a transfer away. Maybe clubs have different motivation or strategies when transferring players. I should consider how these factors affect team development and player performance.

The article also delves into the scoring system with PL30 stats and recent changes like two matches for draws counting as a sixth goal each. I'm thinking, is it worth following this detailed tracking? Maybe for someone tracking their own league progression later on.

Data visualization aspects are another thing in there—graphs showing stats over time and injuries. This would help anyone visualize trends or player performance without having to compute everything manually. That could make trends easier to spot, but maybe it's more about the article than anything else.

Lastly, some quotes highlight the determination and teamwork within the league. That sets a positive tone for the competition, so perhaps encouraging fan engagement on social media would be beneficial.

Overall, the Premier League article seems comprehensive, covering various aspects from structure, statistics to player transfers. There might be a lot of interesting points or potential missing details that could spark more analysis. Maybe if I started by looking at one section and built from there, it'd be easier to follow all the components.
</think>

**Expanded Understanding of the Premier League (2013 onwards)**

The Premier League is a significant football competition in England, particularly for high school teams starting from 2013. Here’s an organized recap of its key aspects and analysis:

### Season Structure:
- **Year-End Adjustments:** The league began with two teams on the field ( xlB like FC United and FC Solon), which was later expanded to include three teams.
- **Division Changes:** Progressions between divisions were made at specific times, such as 18 Feb 2017 for second division, 26 Oct 2017 for third division, and 5 Dec 2017 for fourth division.

### Key Stats:
- **Home and Away Goals:** Detailed statistics showing average home and away goals per match.
- **Top Players:** Mentioned players with high stats, such as Kevin De Bruck, David objectively.
- **Recent Changes:** Emphasis on PL30 scoring system details.

### Club Statistics:
- Each club's performance metrics are provided but not emphasized for deep comparisons unless data-driven approaches are considered.

### Transfer News:
- **Transfer Flexibility:** Major transfers allowed to any team, with fan questions exploring these dynamics.
- **Player Development:** Mentioned teams like Arsenal and Manchester United gaining more players, suggesting they perform well under coaches.

### Data Visualizations:
- **Trend Graphs:** Plottings of statistics over seasons can show progression trends.
- **Injury Tracking:** Provides visibility on player injuries, crucial for fan engagement.

### Social Media Engagement:
- **Promotions:** Encourages fans to follow on social media for updates and tips, enhancing engagement.

### Factor Influencing Transfers:
- Factors include club motivation, strategies during transfers, financial incentives, etc., which shape transfer destinations.

### Competitive Dynamics:
- Clear performance metrics among teams show intensity. Clubs like club Manchester City dominate the top tier with high stats.

The Premier League article offers a comprehensive overview, covering structure, player statistics, fixture tracking, transfer details, data visualization, and social engagement. For deeper analysis, starting with one section and expanding from there can enhance understanding of all aspects comprehensively.